In [1]:
import sys
sys.path.append('../src')

%load_ext autoreload
%autoreload 1

from models.EDA import find_domain_signatures
%aimport models.EDA

In [2]:
import pandas as pd

In [3]:
DATA_PATH = '../data/'
DATA_INTERIM_PATH = DATA_PATH + 'interim/'
train = pd.read_csv(DATA_INTERIM_PATH + 'train_c.csv')

In [4]:
domains = train['domain'].value_counts().keys()
domains

Index(['foxbusiness', 'abqjournal', 'apnews', 'pri', 'newsline',
       'counterpunch', 'motherjones', 'truthdig', 'dailywire', 'thedailybeast',
       ...
       'abc11', 'newsfrompolitics', 'conservativedeplorable', 'freedomfirst',
       'theamericancause', 'latimes', 'washingtonmonthly', 'occupydemocrats',
       'revealingpolitics', 'anncoulter'],
      dtype='object', length=169)

Initialize empty domain keywords list and iterate

In [8]:
signatures = {}
for domain in domains:
    sig = find_domain_signatures(train, domain, should_print=False)
    signatures[domain] = sig

In [9]:
signatures['dcclothesline']

{'according': 0.45,
 'american': 0.411,
 'com': 0.431,
 'courtesy': 0.497,
 'going': 0.411,
 'government': 0.439,
 'just': 0.613,
 'know': 0.428,
 'like': 0.537,
 'new': 0.474,
 'people': 0.668,
 'said': 0.608,
 'state': 0.456,
 'time': 0.547,
 'world': 0.405,
 'year': 0.416,
 'years': 0.404}

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english',binary=True)
X = vectorizer.fit_transform(train['article_text'])

In [11]:
features = vectorizer.get_feature_names()
features.index('according')

74267

In [12]:
X[:,701867].sum()/train.shape[0]

0.58358875

In [13]:
domain = 'truthdig'
features = vectorizer.get_feature_names()
diffs = {}
for key in signatures[domain].keys():
    index = features.index(key)
    percent = X[:,index].sum()/train.shape[0]
    diff = round(float(signatures[domain][key]) - percent, 3)
    diffs[key] = diff

In [23]:
from models.EDA import filter_common_signatures
%aimport models.EDA

In [142]:
diffs = filter_common_signatures(signatures, vectorizer, X)

In [143]:
for key in list(diffs.keys())[0:30]:
    
    print(key, diffs[key])

foxbusiness {'advertisement': 0.577, 'continue': 0.749, 'reading': 0.822}
abqjournal {'advertisement': 0.755}
apnews {'ap': 0.814}
pri {}
newsline {}
counterpunch {}
motherjones {}
truthdig {}
dailywire {}
thedailybeast {'beast': 0.605, 'daily': 0.555, 'finish': 0.519, 'smart': 0.521, 'speedy': 0.527, 'stories': 0.502, 'summary': 0.521}
poynter {}
baptistnews {'baptist': 0.696, 'church': 0.699}
nbcnews {}
reuters {'10': 0.51, '2016': 0.614, '2017': 0.702, '2018': 0.921, 'according': 0.544, 'added': 0.513, 'additional': 0.787, 'april': 0.602, 'based': 0.521, 'biggest': 0.524, 'billion': 0.553, 'chief': 0.61, 'china': 0.547, 'comment': 0.563, 'companies': 0.527, 'company': 0.639, 'coverage': 0.677, 'david': 0.529, 'did': 0.508, 'donald': 0.684, 'editing': 0.964, 'expected': 0.556, 'february': 0.548, 'file': 0.802, 'friday': 0.523, 'global': 0.545, 'government': 0.575, 'group': 0.555, 'images': 0.686, 'including': 0.612, 'international': 0.502, 'jan': 0.658, 'january': 0.538, 'london': 0.

In [144]:
domain = 'foxbusiness'
df = train[train['domain']==domain]
df.reset_index(inplace=True)
df['article_text'] = df['article_text'].str.lower()

/Users/jmill/anaconda3/envs/SemEval2019-4/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [147]:
word = 'opens a new window'
df[(df['article_text'].str.contains(word))].reset_index()['article_text'].replace(word, word.upper()).shape

(43600,)

In [148]:
df.shape

(116106, 14)

In [155]:
sample = df.sample(n=1000)
vectorizer = CountVectorizer(stop_words='english',binary=True,ngram_range=(2,2))
X = vectorizer.fit_transform(sample['article_text'])

In [192]:
sums = X.sum(axis=0)

In [ ]:
feature_names = vectorizer.get_feature_names()

In [198]:
import numpy as np

thresh = 0.15
max_indices = np.argwhere(sums >= thresh*df.shape[0])

In [211]:
for index in max_indices:
    feature = feature_names[index[1]]
    print(feature)
    print(round(sums[0,index[1]]/df.shape[0], 3))

click opens
0.167
continue reading
1.0
disclosure policy
0.264
dow jones
0.173
fool disclosure
0.264
image source
0.176
motley fool
0.264
new window
0.376
opens new
0.376
policy opens
0.251
position stocks
0.167
stocks mentioned
0.168
wall street
0.173


In [193]:
feature_names[2149224]

TypeError: list indices must be integers or slices, not tuple

In [230]:
sums.shape

(1, 252903)

In [225]:
domain = 'thedailybeast'
uni_sigs = find_domain_signatures(train, domain)
bi_sigs = find_domain_signatures(train, domain, thresh=0.15, ngram_range=(2,2))

{'beast': 0.63, 'daily': 0.647, 'day': 0.677, 'don': 0.698, 'just': 0.656, 'know': 0.67, 'like': 0.692, 'need': 0.631, 'new': 0.654, 'news': 0.651, 'people': 0.604, 'start': 0.605, 'time': 0.622}
{'beast speedy': 0.538, 'daily beast': 0.607, 'day stories': 0.538, 'finish day': 0.538, 'know don': 0.538, 'need know': 0.539, 'new york': 0.305, 'news need': 0.538, 'smart summary': 0.538, 'speedy smart': 0.538, 'start finish': 0.538, 'stories daily': 0.538, 'summary news': 0.538, 'united states': 0.165, 'year old': 0.192}


In [234]:
train[(train['domain']=='thedailybeast') & (train['article_text'].str.contains('speedy'))].reset_index()['article_text'][0]

"GOP?s Top Campaign Whisperers for 2012 Presidential Candidates Mitt Romney?s economic advisers are as squarely establishment as he is. This is reassuring to old-school conservatives like New York Times columnist David Brooks, who calls them the ?gold standard of adviser teams,? and it could appeal to middle-of-the-road voters in a general election. They aren?t purists or populists, but then, Republican primary voters already know Romney?s not that guy. The people who whisper advice into a candidate?s ear don?t usually get much attention, but they were a hot topic at Tuesday?s GOP debate on the economy. Asked whom they?d rely on for economic guidance, the candidates gave answers ranging from ?the American people? (Herman Cain) to ?my own father? (Jon Huntsman) to pioneering conservative economist Milton Friedman (who is unavailable; he died in 2006). That last came from Romney, perhaps trying to soften the impact of who his advisers really are?two Ivy League academics and two Capitol H

In [236]:
train[train['article_text'].str.lower().str.contains(' lta ')]['domain'].value_counts()

apnews                   1434
foxbusiness              1082
abqjournal                552
counterpunch              527
reuters                   326
newsline                  258
pri                       251
calwatchdog               215
motherjones               176
truthdig                  149
thedailybeast             117
nbcnews                    78
consortiumnews             66
ivn                        45
chicagoreporter            43
washingtonblade            43
baptistnews                41
dailywire                  37
leftvoice                  35
poynter                    32
eppc                       30
natmonitor                 29
dissentmagazine            27
mintpressnews              24
thegoldwater               23
intrepidreport             21
reviewjournal              19
billmoyers                 18
fury                       17
equalvoiceforfamilies      15
                         ... 
wordpress                   2
bulletsfirst                2
legalinsur

In [238]:
train[train['domain']=='foxbusiness' & (train['article_text'].str.lower().str.contains(' lta '))].reset_index()['article_text'][0]

TypeError: cannot compare a dtyped [bool] array with a scalar of type [bool]